In [1]:
# === Imports ===
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import tensorflow as tf
import random

from tensorflow.keras.layers import (Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense, Dropout, MaxPooling1D, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import (accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score)
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR
import torch.optim as optim
from tqdm import tqdm
import gc


In [2]:
# === Custom Dataset class for PyTorch ===
class MyDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = np.array(label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = torch.tensor(self.data[index], dtype=torch.float32)
        y = torch.tensor(self.label[index], dtype=torch.float32).unsqueeze(0)
        return x, y


In [ ]:
class MyConv1dPadSame(nn.Module):
    """1D convolution with same padding."""
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super().__init__()
        self.stride = stride
        self.kernel_size = kernel_size
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, groups=groups)

    def forward(self, x):
        in_len = x.shape[-1]
        out_len = (in_len + self.stride - 1) // self.stride
        pad_total = max(0, (out_len - 1) * self.stride + self.kernel_size - in_len)
        pad_left = pad_total // 2
        pad_right = pad_total - pad_left
        x = F.pad(x, (pad_left, pad_right))
        return self.conv(x)


class MyMaxPool1dPadSame(nn.Module):
    """1D max pooling with same padding."""
    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = kernel_size
        self.pool = nn.MaxPool1d(kernel_size=kernel_size)

    def forward(self, x):
        in_len = x.shape[-1]
        pad_total = max(0, self.kernel_size - 1)
        pad_left = pad_total // 2
        pad_right = pad_total - pad_left
        x = F.pad(x, (pad_left, pad_right))
        return self.pool(x)
        

In [ ]:
class BasicBlock(nn.Module):
    """Residual block with optional downsampling."""
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, downsample, use_bn, use_do, is_first_block=False):
        super().__init__()
        self.downsample = downsample
        self.out_channels = out_channels
        self.in_channels = in_channels

        self.bn1 = nn.BatchNorm1d(in_channels) if use_bn else nn.Identity()
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(0.5) if use_do else nn.Identity()
        self.conv1 = MyConv1dPadSame(in_channels, out_channels, kernel_size, stride if downsample else 1, groups)

        self.bn2 = nn.BatchNorm1d(out_channels) if use_bn else nn.Identity()
        self.relu2 = nn.ReLU()
        self.do2 = nn.Dropout(0.5) if use_do else nn.Identity()
        self.conv2 = MyConv1dPadSame(out_channels, out_channels, kernel_size, 1, groups)

        self.pool = MyMaxPool1dPadSame(stride) if downsample else nn.Identity()

    def forward(self, x):
        identity = self.pool(x)
        out = self.bn1(x)
        out = self.relu1(out)
        out = self.do1(out)
        out = self.conv1(out)

        out = self.bn2(out)
        out = self.relu2(out)
        out = self.do2(out)
        out = self.conv2(out)

        if self.out_channels != self.in_channels:
            identity = identity.transpose(1, 2)
            pad = self.out_channels - self.in_channels
            identity = F.pad(identity, (0, pad))
            identity = identity.transpose(1, 2)

        return out + identity


In [ ]:
class ResNet1D(nn.Module):
    """1D ResNet for regression."""
    def __init__(self, in_channels, base_filters, kernel_size, stride, groups, n_block,
                 downsample_gap=2, increasefilter_gap=2, use_bn=True, use_do=True):
        super().__init__()

        self.first_conv = MyConv1dPadSame(in_channels, base_filters, kernel_size, stride=1)
        self.first_bn = nn.BatchNorm1d(base_filters)
        self.first_relu = nn.ReLU()

        # Build residual blocks
        self.blocks = nn.ModuleList()
        filters = base_filters
        in_ch = base_filters

        for i in range(n_block):
            if i != 0 and i % increasefilter_gap == 0:
                filters *= 2

            out_ch = filters
            downsample = (i != 0 and i % downsample_gap == 0)

            block = BasicBlock(
                in_channels=in_ch,
                out_channels=out_ch,
                kernel_size=kernel_size,
                stride=stride,
                groups=groups,
                downsample=downsample,
                use_bn=use_bn,
                use_do=use_do,
                is_first_block=(i == 0)
            )
            self.blocks.append(block)
            in_ch = out_ch

        # Final layers
        self.final_bn = nn.BatchNorm1d(filters)
        self.final_relu = nn.ReLU()
        self.fc = nn.Linear(filters, 1)

    def forward(self, x):
        x = self.first_conv(x)
        x = self.first_bn(x)
        x = self.first_relu(x)
        for block in self.blocks:
            x = block(x)
        x = self.final_bn(x)
        x = self.final_relu(x)
        x = x.mean(-1)  # Global average pooling
        return self.fc(x)


In [3]:
def run_training(X_train, y_train, X_val, y_val, X_test, y_test, epochs=20):
    """
    Trains the ResNet1D model for regression using MAE loss and early stopping.
    """

    best_val_mae = float("inf")
    patience_counter = 5
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Convert numpy to float tensors
    y_train, y_val, y_test = map(lambda arr: arr.astype(np.float32), [y_train, y_val, y_test])

    train_loader = DataLoader(MyDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader   = DataLoader(MyDataset(X_val, y_val), batch_size=64, shuffle=False)
    test_loader  = DataLoader(MyDataset(X_test, y_test), batch_size=64, shuffle=False)

    model = ResNet1D(
        in_channels=X_train.shape[1],
        base_filters=16,
        kernel_size=16,
        stride=1,
        groups=1,
        n_block=8,
        downsample_gap=2,
        increasefilter_gap=2,
        use_bn=True,
        use_do=False
    ).to(device)

    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

    for epoch in range(epochs):
        model.train()
        train_loss, train_preds, train_targets = 0, [], []

        for X, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_preds.extend(outputs.detach().cpu().numpy())
            train_targets.extend(y.cpu().numpy())

        # === Validation ===
        model.eval()
        val_loss, val_preds, val_targets = 0, [], []
        with torch.no_grad():
            for X, y in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False):
                X, y = X.to(device), y.to(device)
                outputs = model(X)
                loss = criterion(outputs, y)
                val_loss += loss.item()
                val_preds.extend(outputs.cpu().numpy())
                val_targets.extend(y.cpu().numpy())

        # Convert predictions to arrays and inverse-transform
        train_preds, val_preds = map(np.array, [train_preds, val_preds])
        train_targets, val_targets = map(np.array, [train_targets, val_targets])

        train_preds_rescaled = target_scaler.inverse_transform(train_preds.reshape(-1, 1)).flatten()
        train_targets_rescaled = target_scaler.inverse_transform(train_targets.reshape(-1, 1)).flatten()
        val_preds_rescaled = target_scaler.inverse_transform(val_preds.reshape(-1, 1)).flatten()
        val_targets_rescaled = target_scaler.inverse_transform(val_targets.reshape(-1, 1)).flatten()

        train_mae = mean_absolute_error(train_targets_rescaled, train_preds_rescaled)
        val_mae = mean_absolute_error(val_targets_rescaled, val_preds_rescaled)

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        if val_mae < best_val_mae:
            best_val_mae = val_mae
            torch.save(model.state_dict(), "best_resnet1d_model.pth")
            print(f"New best model saved with Val MAE: {val_mae:.4f}")
            patience = 0
        else:
            patience += 1
            if patience >= patience_counter:
                print("Early stopping triggered.")
                break

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Train MAE: {train_mae:.4f} | Val MAE: {val_mae:.4f}")

    torch.save(model.state_dict(), "resnet1d_model.pth")
    print("Model saved as 'resnet1d_model.pth'")


In [4]:
train_csv, val_csv, test_csv = "4x4_train.csv", "4x4_val.csv", "4x4_test.csv"
dfs = [pd.read_csv(f) for f in (train_csv, val_csv, test_csv)]

# Drop unnecessary columns if present
for df in dfs:
    df.drop(columns=["apple_type", "hdr"], inplace=True, errors="ignore")

# Split features/targets
X_train, y_train = dfs[0].drop(columns=["apple_content"]), dfs[0]["apple_content"]
X_val,   y_val   = dfs[1].drop(columns=["apple_content"]), dfs[1]["apple_content"]
X_test,  y_test  = dfs[2].drop(columns=["apple_content"]), dfs[2]["apple_content"]

# === Scaling ===
scaler = MinMaxScaler()
X_train, X_val, X_test = map(lambda X: scaler.fit_transform(X) if X is X_train else scaler.transform(X), [X_train, X_val, X_test])

target_scaler = MinMaxScaler()
y_train = target_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_val   = target_scaler.transform(y_val.values.reshape(-1, 1)).flatten()
y_test  = target_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

# Reshape to (N, C, L)
for arr in (X_train, X_val, X_test):
    arr.shape = (arr.shape[0], 1, arr.shape[1])


In [5]:
# === Free up GPU memory ===
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
run_training(X_train, y_train, X_val, y_val, X_test, y_test, epochs=150)


In [9]:
model = ResNet1D(
    in_channels=X_train.shape[1],
    base_filters=16,
    kernel_size=16,
    stride=1,
    groups=1,
    n_block=8,
    downsample_gap=2,
    increasefilter_gap=2,
    use_bn=True,
    use_do=False
).to(device)

model.load_state_dict(torch.load("best_resnet1d_model.pth"))
test_loader = DataLoader(MyDataset(X_test, y_test), batch_size=16, shuffle=False)

model.eval()
test_preds, test_targets = [], []
with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device).unsqueeze(1)
        outputs = model(X)
        test_preds.extend(outputs.cpu().numpy())
        test_targets.extend(y.cpu().numpy())

test_preds = np.array(test_preds).squeeze()
test_targets = np.array(test_targets).squeeze()

test_preds_rescaled = target_scaler.inverse_transform(test_preds.reshape(-1, 1)).flatten()
test_targets_rescaled = target_scaler.inverse_transform(test_targets.reshape(-1, 1)).flatten()

test_rmse = np.sqrt(mean_squared_error(test_targets_rescaled, test_preds_rescaled))
test_mae  = mean_absolute_error(test_targets_rescaled, test_preds_rescaled)
test_r2   = r2_score(test_targets_rescaled, test_preds_rescaled)

print(f"RMSE: {test_rmse:.4f} | MAE: {test_mae:.4f} | R²: {test_r2:.4f}")


In [13]:
print("\nSample Predictions vs. Actual Values:")
random_indices = random.sample(range(len(test_preds_rescaled)), 25)

for i in random_indices:
    print(f"Target: {test_targets_rescaled[i]:.2f} | Predicted: {test_preds_rescaled[i]:.2f}")
